#### Mating data process pipeline

In [9]:
import sys
sys.path.append("../")
import os

import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange

from cellmating.utils import file_traverse, load_from_pkl
from cellmating.io import imread

from script_wlli.mating_feature_extract import extract_mating_features, stack_features

In [2]:
root = "/media/wlli/T7/data/mating_all/"
file_list = file_traverse(root, file_regular=r".*/selected/.*.tif")
print("totle images: ", len(file_list))

totle images:  195


In [3]:
from script_wlli.utils import load_configs
config = load_configs("./config_mating.yml")

In [4]:
all_images_feature_over_time = {}
for i in trange(0, 3):
    file_name = file_list[i]
    try:
        image = imread(file_name)
        g = load_from_pkl(file_name[:-4]+".network")
        key = int(file_name.split("_")[-1][:-4])
        key = np.unique(image[-1,-1][image[-1,-2] == key])[0]%1000
        # extract features
        data = extract_mating_features(image, g, key)
        # stack features
        all_images_feature_over_time = stack_features(all_images_feature_over_time, data, os.path.basename(file_name), config=config)
    except Exception as error:
    # handle the exception
        print(file_name," : ", error)

100%|█████████████████████████████████████████████| 3/3 [00:52<00:00, 17.66s/it]


In [ ]:
# dump_to_pkl(all_images_feature_over_time,"./all_time_new.obj")
# for key in all_images_feature_over_time:
#     data = all_images_feature_over_time[key]
#     data.to_csv("./csv/"+key+".csv")